# Atom feed of tweeted new preprints by subject area

Preprints published in the past seven days tweeted at least three times as atom feed, grouped by subject area.

In [18]:
import sys
sys.path.append('../')

import ast
import pandas as pd
from datetime import datetime, date
from feedgen.feed import FeedGenerator

In [19]:
subject_areas = ['covid', 'biochemistry-cell-biology', 'ecology', 'evolutionary-biology', 'genetics', 'microbiology', 'physiology', 'plant-biology', 'zoology','other-biological-sciences']
preprints = pd.read_csv('../preprint_tweets/1101/preprint_tweets_' + str(date.today().strftime('%Y-%m-%d')) + '.csv', index_col=0)

# filter by section, and group all COVID-related preprints together
subsets = {}
subsets['covid'] = preprints[preprints['covid'] == True]
subsets['biochemistry-cell-biology'] = preprints[(preprints['subject-area'] == 'Biochemistry') | (preprints['subject-area'] == 'Cell Biology') & (preprints['covid'] == False)]
subsets['ecology'] = preprints[(preprints['subject-area'] == 'Ecology') & (preprints['covid'] == False)]
subsets['evolutionary-biology'] = preprints[(preprints['subject-area'] == 'Evolutionary Biology') & (preprints['covid'] == False)]
subsets['genetics'] = preprints[(preprints['subject-area'] == 'Genetics') & (preprints['covid'] == False)]
subsets['microbiology'] = preprints[(preprints['subject-area'] == 'Microbiology') & (preprints['covid'] == False)]
subsets['physiology'] = preprints[(preprints['subject-area'] == 'Physiology') & (preprints['covid'] == False)]
subsets['plant-biology'] = preprints[(preprints['subject-area'] == 'Plant Biology') & (preprints['covid'] == False)]
subsets['zoology'] = preprints[(preprints['subject-area'] == 'Zoology') & (preprints['covid'] == False)]
subsets['other-biological-sciences'] = preprints[(preprints['subject-area'] != 'Biochemistry') & (preprints['subject-area'] != 'Cell Biology') & (preprints['subject-area'] != 'Ecology') & (preprints['subject-area'] != 'Evolutionary Biology') & (preprints['subject-area'] != 'Genetics') & (preprints['subject-area'] != 'Microbiology') & (preprints['subject-area'] != 'Physiology') & (preprints['subject-area'] != 'Plant Biology') & (preprints['subject-area'] != 'Zoology') & (preprints['covid'] == False)]
subsets['other-biological-sciences'].head(50)

,doi,tweets,archive,subject-area,covid,title,authors,abstract,posted
4,10.1101/2021.07.23.453379,57,bioRxiv,Bioinformatics,False,"AlphaPept, a modern and open framework for MS-...","[{'name': 'Maximilian T. Strauss'}, {'name': '...","<p>In common with other omics technologies, ma...",2021-07-26
5,10.1101/2021.07.23.453492,52,bioRxiv,Scientific Communication and Education,False,Delineating Medical Education: Bibliometric Re...,"[{'name': 'Lauren A. Maggio'}, {'name': 'Anton...",<sec><title>Background</title><p>The field of ...,2021-07-26
6,10.1101/2021.07.23.453605,51,bioRxiv,Genomics,False,Identifying cell-state associated alternative ...,"[{'name': 'Carlos F. Buen Abad Najar'}, {'name...",<p>Alternative splicing shapes the transcripto...,2021-07-24
9,10.1101/2021.07.23.453478,44,bioRxiv,Biophysics,False,Mechanosensitivity of nucleocytoplasmic transport,"[{'name': 'Ion Andreu'}, {'name': 'Ignasi Gran...",<p>Mechanical force controls fundamental cellu...,2021-07-24
10,10.1101/2021.07.23.453573,43,bioRxiv,Neuroscience,False,Strong and opponent contributions of dorsomedi...,"[{'name': 'Scott S. Bolkan'}, {'name': 'Iris R...",<p>A classic view of the striatum holds that a...,2021-07-25
11,10.1101/2021.07.22.453458,41,bioRxiv,Cancer Biology,False,Epithelial-mesenchymal plasticity determines e...,"[{'name': 'Patrick Aouad'}, {'name': 'Yueyun Z...",<p>Estrogen receptor α-positive (ER+) breast c...,2021-07-26
16,10.1101/2021.07.21.453258,30,bioRxiv,Neuroscience,False,Retrosplenial cortex is necessary for spatial ...,"[{'name': 'Ana Carolina Bottura de Barros'}, {...",<p>Latent learning occurs when associations ar...,2021-07-23
17,10.1101/2021.07.27.453460,29,bioRxiv,Developmental Biology,False,Extracellular mechanical forces drive endocard...,"[{'name': 'Julien Vermot'}, {'name': 'Helene V...",<p>Organ morphogenesis involves dynamic change...,2021-07-27
18,10.1101/2021.07.26.453816,23,bioRxiv,Neuroscience,False,Synaptic Mechanisms of Top-Down Control by The...,"[{'name': 'Hannah M. Oberle'}, {'name': 'Alex ...",<p>Corticofugal projections to evolutionarily ...,2021-07-26
19,10.1101/2021.07.26.453747,23,bioRxiv,Bioinformatics,False,ABlooper: Fast accurate antibody CDR loop stru...,"[{'name': 'Brennan Abanades'}, {'name': 'Guy G...",<p>Antibodies are a key component of the immun...,2021-07-26


In [20]:

for subject_area in subject_areas:
    fg = FeedGenerator()
    fg.id('https://front-matter.io/')
    if subject_area == 'covid':
        fg.title(f'Popular BioRxiv/medRxiv COVID preprints posted in the last week')
    else:
        fg.title(f'Popular BioRxiv {subject_area.replace("_", " ")} preprints posted in the last week')
    fg.author( {'name':'Martin Fenner','email':'martin@front-matter.io'} )
    fg.link( href='https://front-matter.io', rel='alternate' )
    fg.link( href=f'https://front-matter.io/{subject_area}/feed.xml', rel='self' )
    fg.language('en')

    for index, row in subsets[subject_area].iterrows():
        fe = fg.add_entry(order='append')
        fe.title(row['title'] + f" ({row['tweets']} tweets)")
        fe.author(ast.literal_eval(row['authors']))
        fe.id('https://doi.org/' + row['doi'])
        fe.link({'href': 'https://doi.org/' + row['doi'], 'title': row['title'] + f" ({row['tweets']} tweets)"})
        fe.description(row['abstract'])
        fe.category({ 'term': row['subject-area']})
        fe.published(datetime.fromisoformat(row['posted'] + 'T00:00:00+00:00'))
                 
    fg.atom_file(f'{subject_area}/feed.xml')